In [1]:
import logging
import os
from glob import glob
from rasterio.features import sieve
import rasterio
import numpy as np
import argparse


def main():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('-dir', '--directory', required=True, help='Specify directory that contains raster files')
    parser.add_argument('-s', '--size', required=False, type=int, default=15, help='minimum size of sieve kernel')
    parser.add_argument('-conn', '--connectivity', required=False, type=int, default=4,
                        help='set connectivity to 4 or 8')
    parser.add_argument('-dt', '--dtype', required=False, type=str, default='uint16', help='raster data type')
    parser.add_argument('-f', '--from_values', required=False, nargs='+', type=int, default=[0, 1, 2, 3, 4, 5, 7],
                        help='set from values to reclassify')
    parser.add_argument('-t', '--to_values', required=False, nargs='+', type=int,
                        default=[998, 41, 21, 81, 11, 81, 998],
                        help='set to values to reclassify')
    args = parser.parse_args()
    saveDir = args.directory
    files = glob(os.path.join(saveDir, '*.tif'))
    logging.basicConfig(filename=os.path.join(saveDir, "log.txt"), filemode='a')
    for file in files:
        try:
            raster = rasterio.open(file)
            array = raster.read(1).astype(args.dtype)
            profile = raster.profile
            profile.update({"dtype": args.dtype})
            out = os.path.join(saveDir, os.path.basename(file)[:-4] + '_sieve_reclass.tif')
            array = sieve(array, size=args.size, connectivity=args.connectivity)
            for fv, tv in zip(args.from_values, args.to_values):
                array = np.where(array == fv, tv, array)
            with rasterio.open(out, 'w', **profile) as out_r:
                out_r.write(array, 1)
                print(f"Finished sieving & reclassifying: \n{file} \nto: {out}")
        except Exception as e:
            print(f"Failed to sieve & reclassify: \n{file} \n{e}")
            logging.exception(f'exception in {file} \n{e}')
            continue


if __name__ == '__main__':
    main()


ImportError: DLL load failed while importing _base: The specified module could not be found.